In [1]:
import os
from pyprojroot import here
os.environ["HF_HOME"] = str(here("cache/huggingface"))
import lightning as L
from importlib import reload
import sys
sys.path.append(str(here()))
from models import model_definitions as MD
import torch
import pandas as pd
reload(MD)
torch.set_float32_matmul_precision('medium')
data_module_3 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_3.parquet")))
data_module_7 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_7.parquet")))
data_module_28 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_28.parquet")))
metrics = MD.MetricsCallback()

def make_trainer():
  return L.Trainer(
    max_epochs=15,
    max_time={"minutes": 10},
    callbacks=[
      # metrics,
    ],
    deterministic=True,
    default_root_dir=here("cache/lightning"),
    enable_checkpointing=True,
    fast_dev_run=False,
    accelerator="auto", 
  )

trainer = make_trainer()
trainer

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


# Distil Bert finetuned (cross entropy loss)

## 3 classes

In [2]:
reload(MD)
distil_3 = MD.DistilBertFinetune(n_emotions=3)
if False:    
  trainer.fit(
    distil_3,
    datamodule=data_module_3,
  )
else:
  distil_3 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_3_14epchs.ckpt"),
    n_emotions=3,
  )
out = trainer.test(
  distil_3,
  datamodule=data_module_3,
)
del distil_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_work

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.4721657335758209
    test_expectedNDCG       0.6351519227027893
    test_f1_interest        0.5440922379493713
      test_f1_stand         0.5834985971450806
        test_nDGC           0.9334059357643127
        test_rmse           0.25550487637519836
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.4721657335758209,
  'test_f1_stand': 0.5834985971450806,
  'test_f1_interest': 0.5440922379493713,
  'test_rmse': 0.25550487637519836,
  'test_nDGC': 0.9334059357643127,
  'test_expectedNDCG': 0.6351519227027893}]

## 7 classes

In [3]:
reload(MD)
distil_7 = MD.DistilBertFinetune(n_emotions=7)
if False:    
  trainer.fit(
    distil_7,
    data_module=data_module_7,
  )
else:
  distil_7 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_7_14epchs.ckpt"),
    n_emotions=7,
  )
out = trainer.test(
  distil_7,
  datamodule=data_module_7
)
del distil_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.2804671823978424
    test_expectedNDCG       0.4182862639427185
    test_f1_interest        0.2835002541542053
      test_f1_stand         0.23864038288593292
        test_nDGC           0.8956641554832458
        test_rmse            0.186847522854805
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.2804671823978424,
  'test_f1_stand': 0.23864038288593292,
  'test_f1_interest': 0.2835002541542053,
  'test_rmse': 0.186847522854805,
  'test_nDGC': 0.8956641554832458,
  'test_expectedNDCG': 0.4182862639427185}]

## 28 classes

In [7]:
reload(MD)
distil_28 = MD.DistilBertFinetune(n_emotions=28)
trainer = make_trainer()
if False:    
  trainer.fit(
    distil_28,
    datamodule=data_module_28,
  )
else:
  distil_28 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_28_11epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_28,
  datamodule=data_module_28,
)
del distil_28
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_approxNDCG        0.6911609172821045
   test_cross_entropy       0.11740745604038239
    test_f1_interest        0.09376711398363113
      test_f1_stand         0.0886179506778717
        test_nDGC           0.8142855167388916
        test_rmse           0.11035352200269699
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.11740745604038239,
  'test_f1_stand': 0.0886179506778717,
  'test_f1_interest': 0.09376711398363113,
  'test_rmse': 0.11035352200269699,
  'test_nDGC': 0.8142855167388916,
  'test_approxNDCG': 0.6911609172821045}]

# Distil Bert finetuned (DCG loss)

## 3 classes

In [5]:
reload(MD) # DistilBertFinetuneOnDCG
distil_dcg_3 = MD.DistilBertFinetuneOnDCG(n_emotions=3)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_3,
    datamodule=data_module_3,
  )
else:
  distil_dcg_3 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_dcg_3.ckpt"),
    n_emotions=3,
  ) 
out = trainer.test(
  distil_dcg_3,
  datamodule=data_module_3,
)
del distil_dcg_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                                | Params | Mode 
------------------------------------------------------------------------------
0 | model         | DistilBertForSequenceClassification | 67.0 M | train
1 | sigmoid       | Sigmoid                             | 0      | train
2 | sig_loss      | BCEWithLogit

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.5865071415901184
    test_expectedNDCG       0.6287815570831299
    test_f1_interest        0.45578745007514954
      test_f1_stand         0.7278568744659424
        test_nDGC           0.9333325028419495
        test_rmse           0.33281922340393066
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.5865071415901184,
  'test_f1_stand': 0.7278568744659424,
  'test_f1_interest': 0.45578745007514954,
  'test_rmse': 0.33281922340393066,
  'test_nDGC': 0.9333325028419495,
  'test_expectedNDCG': 0.6287815570831299}]

## 7 classes

In [6]:
reload(MD)
distil_dcg_7 = MD.DistilBertFinetuneOnDCG(n_emotions=7)
trainer = make_trainer()
if True:    
  trainer.fit(
    distil_dcg_7,
    datamodule=data_module_7,
  )
else:
  distil_dcg_7 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_dcg_7.ckpt"),
    n_emotions=7,
  )
  
out = trainer.test(
  distil_dcg_7,
  datamodule=data_module_7,
)
del distil_dcg_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                                | Params | Mode 
------------------------------------------------------------------------------
0 | model         | DistilBertForSequenceClassification | 67.0 M | train
1 | sigmoid       | Sigmoid                             | 0      | train
2 | sig_loss      | BCEWithLogit

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.42345064878463745
    test_expectedNDCG       0.4156492054462433
    test_f1_interest        0.18776972591876984
      test_f1_stand         0.38860616087913513
        test_nDGC           0.8896769881248474
        test_rmse           0.28936758637428284
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.42345064878463745,
  'test_f1_stand': 0.38860616087913513,
  'test_f1_interest': 0.18776972591876984,
  'test_rmse': 0.28936758637428284,
  'test_nDGC': 0.8896769881248474,
  'test_expectedNDCG': 0.4156492054462433}]

## 28 classes

In [20]:
reload(MD)
distil_dcg_28 = MD.DistilBertFinetuneOnDCG(n_emotions=28)
trainer = make_trainer()
if True:
  trainer.fit(
    distil_dcg_28,
    datamodule=data_module_28,
  )
else:
  distil_dcg_28 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_dcg_28.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_dcg_28,
  datamodule=data_module_28,
)
del distil_dcg_28
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type                                | Params | Mode 
----------------------------------------------------------------------------
0 | model       | DistilBertForSequenceClassification | 67.0 M | train
1 | sigmoid     | Sigmoid                             | 0      | train
2 | sig_loss    | BCEWithLogitsLoss     

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [24]:
# test sample
trainer.predict(
  MD.DistilBertFinetuneOnDCG.load_from_checkpoint(r"C:\Users\Plancha\emotion-recognition\cache\lightning\lightning_logs\version_11\checkpoints\epoch=1-step=1614.ckpt", n_emotions=28),
  ["I Love this"],
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[4.1811e-03, 1.0000e+00, 8.3217e-04, 1.9332e-03, 3.4661e-03, 1.5848e-03,
          2.9398e-04, 7.1204e-04, 3.2484e-03, 3.0075e-04, 9.0104e-04, 2.9052e-04,
          1.2698e-03, 4.7179e-04, 2.4381e-04, 2.5546e-04, 4.0613e-04, 1.5460e-04,
          3.6107e-04, 3.1165e-04, 3.5666e-04, 1.5112e-04, 1.7924e-04, 1.4317e-04,
          3.0266e-04, 1.9306e-04, 1.9489e-04, 1.9270e-04]])]